### Pade objective builder function

In [1]:
using Symbolics
using LinearAlgebra
using DynamicPolynomials

Auxilary function for square Frobenius norm of the matrix $\left\| M \right\|_F^2$:

In [2]:
function frobenius_norm2(m)
    return tr(m * m')
end

frobenius_norm2 (generic function with 1 method)

Lindblad master equation is given by
\begin{align}\label{eq:L1}
    \frac{d}{dt}{\rho}(t) = \mathcal{L}\rho(t),
\end{align}

where Lindbladian right hand side operator is given as
\begin{align}
    \mathcal{L} [\rho] = - \frac{i}{\hbar}[H, \rho]+\sum_{j=1}^{\ell}\left[ J_j \rho J_j^\dagger - \frac{1}{2}\left\{ J_j^\dagger J_j, \rho \right\} \right],
\end{align}

\begin{align}
\sum_{i=1}^N {
{\left\|
\rho_{(i)} - \rho_{(i-1)}
- {\mathcal{L}} \left[\int^{i\Delta t}_{(i-1)\Delta t}\rho(t)dt \right] \right\| }^2_F } \rightarrow \operatorname{min}
\end{align}

In [3]:
function lindblad_rhs(ρ, H, J)
    """
    Right hand side of the Lindblad master equation
    """
    return -im*(H * ρ - ρ * H) + J * ρ * J' - (J' * J  * ρ + ρ * J' * J) / 2
    
end

lindblad_rhs (generic function with 1 method)

Terms for the objective for minimization:
\begin{align}
{\rho_{(i)} - \rho_{(i-1)} - \Delta t \mathcal{L} \left[\frac{{\rho}_{(i)} + {\rho}_{(i-1)}}{2} \right]} \rightarrow \operatorname{min}
\end{align}

In [4]:
function pade_obj(ρ::Array{ComplexF64,3}, t, H, J)
    
    obj = 0
    for i in 2:size(ρ,3)
        obj += frobenius_norm2(
            ρ[:, :, i] - ρ[:, :, i-1] 
            - (t[i]-t[i-1])*lindblad_rhs((ρ[:, :, i]+ρ[:, :, i-1])/2, H, J)
        )
    end
    obj = sum(real(coef) * mon for (coef, mon) in zip(coefficients(obj), monomials(obj)))
    return obj
end

function pade_obj(ρ::Vector{Matrix{ComplexF64}}, t, H, J)
    obj = 0
    for i in 2:size(ρ,1)
        obj += frobenius_norm2(
            ρ[i] - ρ[i-1] 
            - (t[i]-t[i-1])*lindblad_rhs((ρ[i]+ρ[i-1])/2, H, J)
        )
    end
    obj = sum(real(coef) * mon for (coef, mon) in zip(coefficients(obj), monomials(obj)))
    return obj
end

pade_obj (generic function with 2 methods)

Consider test example

In [5]:
@polyvar x[1:4]
H = [ 1.0 * x[1]              x[3] + im * x[4]
      x[3] - im * x[4]        x[2]             ]

@polyvar a[1:2, 1:2]
@polyvar b[1:2, 1:2]
A = 1.0 * a + im * b

ρ1 = [.5 -im
     +im  .5]
ρ2 = [.6 0
     0  .4]
ρ3 = [.7 0
     0  .3]
ρ4 = [.9 0
     0  .1]

ρ = [ρ1;;; ρ2;;; ρ3;;; ρ4]
t = [0.0, 0.1, 0.2, 0.3]

obj = pade_obj(ρ, t, H, A)

0.0012500000000000002a₁₋₁⁴ - 0.005000000000000001a₁₋₁³a₂₋₂ - 0.005500000000000001a₁₋₁³b₂₋₁ - 0.005000000000000001a₁₋₁³b₁₋₂ + 0.029799999999999997a₁₋₁²a₂₋₁² - 0.03499999999999999a₁₋₁²a₂₋₁a₁₋₂ + 0.005500000000000001a₁₋₁²a₂₋₁b₁₋₁ - 0.016000000000000004a₁₋₁²a₂₋₁b₂₋₂ + 0.017499999999999998a₁₋₁²a₁₋₂² + 0.005000000000000001a₁₋₁²a₁₋₂b₁₋₁ + 0.005500000000000001a₁₋₁²a₁₋₂b₂₋₂ + 0.0075000000000000015a₁₋₁²a₂₋₂² + 0.016000000000000004a₁₋₁²a₂₋₂b₂₋₁ + 0.014500000000000002a₁₋₁²a₂₋₂b₁₋₂ + 0.0025000000000000005a₁₋₁²b₁₋₁² - 0.005000000000000001a₁₋₁²b₁₋₁b₂₋₂ + 0.029799999999999997a₁₋₁²b₂₋₁² + 0.04499999999999999a₁₋₁²b₂₋₁b₁₋₂ + 0.0225a₁₋₁²b₁₋₂² + 0.0075000000000000015a₁₋₁²b₂₋₂² - 0.04499999999999999a₁₋₁a₂₋₁²a₂₋₂ - 0.005500000000000001a₁₋₁a₂₋₁²b₂₋₁ - 0.005000000000000001a₁₋₁a₂₋₁²b₁₋₂ + 0.04539999999999999a₁₋₁a₂₋₁a₁₋₂a₂₋₂ + 0.020000000000000004a₁₋₁a₂₋₁a₂₋₂b₂₋₂ - 0.07999999999999999a₁₋₁a₂₋₁b₁₋₁b₁₋₂ + 0.055399999999999984a₁₋₁a₂₋₁b₁₋₂b₂₋₂ - 0.025a₁₋₁a₁₋₂²a₂₋₂ - 0.015500000000000003a₁₋₁a₁₋₂²b₂₋₁ - 0.0140000000000

In [6]:
using TSSOS

opt,sol,data = tssos_first(obj, variables(obj));

************************TSSOS************************
TSSOS is launching...
Starting to compute the block structure...
------------------------------------------------------
The sizes of PSD blocks:
[22, 19, 4]
[1, 1, 2]
------------------------------------------------------
Obtained the block structure. The maximal size of blocks is 22.
Assembling the SDP...
There are 267 affine constraints.
Solving the SDP...
Problem
  Name                   :                 
  Objective sense        : max             
  Type                   : CONIC (conic optimization problem)
  Constraints            : 267             
  Cones                  : 0               
  Scalar variables       : 1               
  Matrix variables       : 4               
  Integer variables      : 0               

Optimizer started.
Presolve started.
Linear dependency checker started.
Linear dependency checker terminated.
Eliminator started.
Freed constraints in eliminator : 0
Eliminator terminated.
Eliminator - trie

In [7]:
opt,sol,data = tssos_higher!(data);

Starting to compute the block structure...
No higher TSSOS hierarchy!


In [8]:
opt,sol,data = tssos_higher!(data);

Starting to compute the block structure...
No higher TSSOS hierarchy!
